In [60]:
from psaw import PushshiftAPI
import praw
import pprint
from datetime import datetime, timedelta
import pandas as pd
import os

In [3]:
# Reddit Agent Authorization Data
client_id = "tpammI9-HYB25Q"
secret_token = "aDm9bgjz6Vn1QtePAEEzWNJ5wyforg"
username="AbbieSnoozeAlot"
password="E2zT9vV7GxWtawK"
agent="SrsBot/0.0.1"

reddit = praw.Reddit(
    client_id=client_id,
    client_secret=secret_token,
    user_agent=agent,
    username=username,
    password=password,
)

reddit.read_only = True
api = PushshiftAPI(reddit)

In [158]:
subreddit_list = [
    "wallstreetbets",
#     "options"
]
# subreddit_list = [
#     "options"
# ]

In [159]:
deleted_keywords = ["[deleted]", "[removed]", "", None]
submission_columns = ["name", "text", "score", "upvote_ratio", "created_utc"]
comment_columns = ["submission_name", "id", "text", "score", "created_utc"]

In [160]:
class Data_Handler:
    def __init__(self, subreddit, buffer_size=1000):
        self.subreddit = subreddit
        self.submission_buffer = [] 
        self.comment_buffer = []
        self.buffer_size = buffer_size
        
        self.last_submission_name = None
        self.last_comment_id = None

    def Create_CSV(self, initial_date):
        subreddit=self.subreddit
        submission_csv = f"{subreddit}_submissions.csv"
        comments_csv = f"{subreddit}_comments.csv"

        if not os.path.exists(submission_csv):
            with open(submission_csv,"w") as file:
                file.write(",".join(submission_columns))
                file.write("\n")
                
            with open(comments_csv,"w") as file:
                file.write(",".join(comment_columns))
                file.write("\n")
            
            return initial_date
        else:
            last_submission = pd.read_csv(submission_csv).iloc[-1]
            last_comment = pd.read_csv(comments_csv).iloc[-1]
            
            self.last_submission_name = last_submission["name"]
            self.last_comment_id = last_comment["id"]
            
            last_submission_time = datetime.utcfromtimestamp(last_submission["created_utc"])
            last_comment_time = datetime.utcfromtimestamp(last_comment["created_utc"])
            last_time = last_submission_time if last_submission_time < last_comment_time else last_comment_time
            last_time = last_time.replace(hour=0, minute=0, second=0, microsecond=0)
            return last_time
                


    def Write_Data(self):
        subreddit=self.subreddit
        submission_buffer=self.submission_buffer
        comment_buffer=self.comment_buffer
        buffer_size=self.buffer_size
        
        if buffer_size <= len(submission_buffer):
            
            while self.last_submission_name != None and len(submission_buffer) != 0:
                temp_name = submission_buffer.pop(0)[0] #index 0 is name
#                 print(temp_name, self.last_submission_name)
                if temp_name == self.last_submission_name:
                    print("FOUND IT", temp_name)
                    self.last_submission_name = None
                    
            if len(submission_buffer) != 0:        
                submission_df = pd.DataFrame(submission_buffer, columns = submission_columns)
                submission_df.to_csv(f"{subreddit}_submissions.csv", mode="a", header=False, index=False)
            self.submission_buffer = []
        if buffer_size <= len(comment_buffer):
            
            while self.last_comment_id != None and len(comment_buffer) != 0:
#                 temp_name = comment_buffer[0][0]
                temp_id = comment_buffer.pop(0)[1] #index 1 is id
#                 print(temp_name, temp_id, self.last_comment_id)
                if temp_id == self.last_comment_id:
                    print("FOUND IT", temp_id)
                    self.last_comment_id = None
            
            if len(comment_buffer) != 0:
                comment_df = pd.DataFrame(comment_buffer, columns = comment_columns)
                comment_df.to_csv(f"{subreddit}_comments.csv", mode="a", header=False, index=False)
            self.comment_buffer = []

In [ ]:
initial_date = datetime(2014,1,1)
end_date = datetime(2021,1,1)
delta_time = timedelta(days=1)

buffer_size = 1000
min_num_interactions = 2
for sr in subreddit_list:
    cur_date = initial_date
    data_handler = Data_Handler(sr, buffer_size=buffer_size)
    cur_date = data_handler.Create_CSV(initial_date)
    while cur_date < end_date:
#         if cur_date.day == 0:
#             print(cur_date)
        print(cur_date)
        
        gen = api.search_submissions(after=cur_date, before=cur_date + delta_time,
                                    subreddit=sr,
                                    filter=["url","author", "title", "subreddit", "selftext"])
        
        
        for submission in gen:
            submission_text = submission.selftext
            submission_score = submission.score
            if(submission_text in deleted_keywords or submission_score < min_num_interactions):
                continue
                
            data_handler.submission_buffer.append([submission.name, 
                                      submission_text, 
                                      submission_score, 
                                      submission.upvote_ratio,
                                      submission.created_utc])
            
            submission.comments.replace_more(limit=None)
            for comment in submission.comments:
                comment_text = comment.body
                comment_score = comment.score
                if(comment_text in deleted_keywords or comment_score < min_num_interactions):
                    continue
                
                data_handler.comment_buffer.append([submission.name, 
                                       comment.id, 
                                       comment_text, 
                                       comment_score,
                                       comment.created_utc])
        
        data_handler.Write_Data()
        cur_date += delta_time
    
    data_handler.Write_Data() #Flush remaining buffer

2016-08-05 00:00:00
2016-08-06 00:00:00
2016-08-07 00:00:00
2016-08-08 00:00:00
2016-08-09 00:00:00
2016-08-10 00:00:00
2016-08-11 00:00:00
2016-08-12 00:00:00
2016-08-13 00:00:00
2016-08-14 00:00:00
2016-08-15 00:00:00
2016-08-16 00:00:00
2016-08-17 00:00:00
2016-08-18 00:00:00
2016-08-19 00:00:00
2016-08-20 00:00:00
2016-08-21 00:00:00
2016-08-22 00:00:00
FOUND IT d6qbfgh
2016-08-23 00:00:00
2016-08-24 00:00:00
2016-08-25 00:00:00
2016-08-26 00:00:00
2016-08-27 00:00:00
2016-08-28 00:00:00
2016-08-29 00:00:00
2016-08-30 00:00:00
2016-08-31 00:00:00
2016-09-01 00:00:00
2016-09-02 00:00:00
2016-09-03 00:00:00
2016-09-04 00:00:00
2016-09-05 00:00:00
2016-09-06 00:00:00
2016-09-07 00:00:00
2016-09-08 00:00:00
FOUND IT t3_4w80lr
2016-09-09 00:00:00
2016-09-10 00:00:00
2016-09-11 00:00:00
2016-09-12 00:00:00
2016-09-13 00:00:00
2016-09-14 00:00:00
2016-09-15 00:00:00
2016-09-16 00:00:00
2016-09-17 00:00:00
2016-09-18 00:00:00
2016-09-19 00:00:00
2016-09-20 00:00:00
2016-09-21 00:00:00
2016

/home/mfclinton/anaconda3/lib/python3.8/site-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)


2016-12-17 00:00:00
2016-12-18 00:00:00
2016-12-19 00:00:00
2016-12-20 00:00:00
2016-12-21 00:00:00
2016-12-22 00:00:00
2016-12-23 00:00:00
2016-12-24 00:00:00
2016-12-25 00:00:00
2016-12-26 00:00:00
2016-12-27 00:00:00
2016-12-28 00:00:00
2016-12-29 00:00:00
2016-12-30 00:00:00
2016-12-31 00:00:00
2017-01-01 00:00:00
2017-01-02 00:00:00
2017-01-03 00:00:00
2017-01-04 00:00:00
2017-01-05 00:00:00
2017-01-06 00:00:00
2017-01-07 00:00:00
2017-01-08 00:00:00
2017-01-09 00:00:00
2017-01-10 00:00:00
2017-01-11 00:00:00
2017-01-12 00:00:00
2017-01-13 00:00:00
2017-01-14 00:00:00
2017-01-15 00:00:00
2017-01-16 00:00:00
2017-01-17 00:00:00
2017-01-18 00:00:00
2017-01-19 00:00:00
2017-01-20 00:00:00
2017-01-21 00:00:00
2017-01-22 00:00:00
2017-01-23 00:00:00
2017-01-24 00:00:00
2017-01-25 00:00:00
2017-01-26 00:00:00
2017-01-27 00:00:00
2017-01-28 00:00:00
2017-01-29 00:00:00
2017-01-30 00:00:00
2017-01-31 00:00:00
2017-02-01 00:00:00
2017-02-02 00:00:00
2017-02-03 00:00:00
2017-02-04 00:00:00


/home/mfclinton/anaconda3/lib/python3.8/site-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 520
  warnings.warn("Got non 200 code %s" % response.status_code)


2017-03-22 00:00:00
2017-03-23 00:00:00
2017-03-24 00:00:00
2017-03-25 00:00:00
2017-03-26 00:00:00
2017-03-27 00:00:00
2017-03-28 00:00:00
2017-03-29 00:00:00
2017-03-30 00:00:00
2017-03-31 00:00:00
2017-04-01 00:00:00
2017-04-02 00:00:00
2017-04-03 00:00:00
2017-04-04 00:00:00
2017-04-05 00:00:00
2017-04-06 00:00:00
2017-04-07 00:00:00
2017-04-08 00:00:00
2017-04-09 00:00:00
2017-04-10 00:00:00
2017-04-11 00:00:00
2017-04-12 00:00:00
2017-04-13 00:00:00
2017-04-14 00:00:00
2017-04-15 00:00:00
2017-04-16 00:00:00
2017-04-17 00:00:00
2017-04-18 00:00:00
2017-04-19 00:00:00
2017-04-20 00:00:00
2017-04-21 00:00:00
2017-04-22 00:00:00
2017-04-23 00:00:00
2017-04-24 00:00:00
2017-04-25 00:00:00
2017-04-26 00:00:00
2017-04-27 00:00:00
2017-04-28 00:00:00
2017-04-29 00:00:00
2017-04-30 00:00:00
2017-05-01 00:00:00
2017-05-02 00:00:00
2017-05-03 00:00:00
2017-05-04 00:00:00
2017-05-05 00:00:00
2017-05-06 00:00:00
2017-05-07 00:00:00
2017-05-08 00:00:00
2017-05-09 00:00:00
2017-05-10 00:00:00


/home/mfclinton/anaconda3/lib/python3.8/site-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 502
  warnings.warn("Got non 200 code %s" % response.status_code)


2017-11-29 00:00:00
2017-11-30 00:00:00
2017-12-01 00:00:00
2017-12-02 00:00:00
2017-12-03 00:00:00
2017-12-04 00:00:00
2017-12-05 00:00:00
2017-12-06 00:00:00
2017-12-07 00:00:00
2017-12-08 00:00:00
2017-12-09 00:00:00
2017-12-10 00:00:00
2017-12-11 00:00:00
2017-12-12 00:00:00
2017-12-13 00:00:00
2017-12-14 00:00:00
2017-12-15 00:00:00
2017-12-16 00:00:00
2017-12-17 00:00:00
2017-12-18 00:00:00
2017-12-19 00:00:00
2017-12-20 00:00:00
2017-12-21 00:00:00
2017-12-22 00:00:00
2017-12-23 00:00:00
2017-12-24 00:00:00
2017-12-25 00:00:00
2017-12-26 00:00:00
2017-12-27 00:00:00
2017-12-28 00:00:00
2017-12-29 00:00:00
2017-12-30 00:00:00
2017-12-31 00:00:00
2018-01-01 00:00:00
2018-01-02 00:00:00


/home/mfclinton/anaconda3/lib/python3.8/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


2018-01-03 00:00:00
2018-01-04 00:00:00
2018-01-05 00:00:00
2018-01-06 00:00:00
2018-01-07 00:00:00
2018-01-08 00:00:00
2018-01-09 00:00:00
2018-01-10 00:00:00
2018-01-11 00:00:00
2018-01-12 00:00:00
2018-01-13 00:00:00
2018-01-14 00:00:00
2018-01-15 00:00:00
2018-01-16 00:00:00
2018-01-17 00:00:00
2018-01-18 00:00:00
2018-01-19 00:00:00
2018-01-20 00:00:00
2018-01-21 00:00:00
2018-01-22 00:00:00
2018-01-23 00:00:00
2018-01-24 00:00:00
2018-01-25 00:00:00
2018-01-26 00:00:00
2018-01-27 00:00:00
2018-01-28 00:00:00
2018-01-29 00:00:00
2018-01-30 00:00:00
2018-01-31 00:00:00
2018-02-01 00:00:00
2018-02-02 00:00:00
2018-02-03 00:00:00
2018-02-04 00:00:00
2018-02-05 00:00:00
2018-02-06 00:00:00
2018-02-07 00:00:00
2018-02-08 00:00:00
2018-02-09 00:00:00
2018-02-10 00:00:00
2018-02-11 00:00:00
2018-02-12 00:00:00
2018-02-13 00:00:00
2018-02-14 00:00:00
2018-02-15 00:00:00
2018-02-16 00:00:00
2018-02-17 00:00:00
2018-02-18 00:00:00
2018-02-19 00:00:00
2018-02-20 00:00:00
2018-02-21 00:00:00
